## Top words from tweets

The following script finds the most frequent words in a set of tweets and creates a bargraph of those words. Many times word frequencie analyses are used to see word frequencies in relation to eachother, but this particular script strips out stopwords so the resulting words shed light on recuring themes.

###  Before we begin
Before we start, you will need to have set up a [Carbonate account](https://kb.iu.edu/d/aolp) in order to access [Research Desktop (ReD)](https://kb.iu.edu/d/apum). You will also need to have access to ReD through the [thinlinc client](https://kb.iu.edu/d/aput). If you have not done any of this, or have only done some of this, but not all, you should go to our [textPrep-Py.ipynb](https://github.com/cyberdh/Text-Analysis/blob/master/textPrep-Py.ipynb) before you proceed further. The textPrep-Py notebook provides information and resources on how to get a Carbonate account, how to set up ReD, and how to get started using the Jupyter Notebook on ReD.   

### Run CyberDH environment
The code in the cell below points to a Python environment specifically for use with the Python Jupyter Notebooks created by Cyberinfrastructure for Digital Humanities. It allows for the use of the different packages in our notebooks and their subsequent data sets.

##### Packages
- **sys:** Provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter. It is always available.
- **os:** Provides a portable way of using operating system dependent functionality.

#### NOTE: This cell is only for use with Research Desktop. You will get an error if you try to run this cell on your personal device!!

In [1]:
import sys
import os
sys.path.insert(0,"/N/u/cyberdh/Carbonate/dhPyEnviron/lib/python3.6/site-packages")
os.environ["NLTK_DATA"] = "/N/u/cyberdh/Carbonate/dhPyEnviron/nltk_data"

### Include necessary packages for notebook 

Python's extensibility comes in large part from packages. Packages are groups of functions, data, and algorithms that allow users to easily carry out processes without recreating the wheel. Some packages are included in the basic installation of Python, others created by Python users are available for download. Make sure to have the following packages installed before beginning so that they can be accessed while running the scripts.

In your terminal, packages can be installed by typing `pip install nameofpackage --user`. However, since you are using ReD and our Python environment, you will not need to install any of the packages below to use this notebook. Anytime you need to make use of a package, however, you need to import it so that Python knows to look in these packages for any functions or commands you use. Below is a brief description of the packages we are using in this notebook:   


- **nltk:** Platform for building Python programs to work with human language data.

- **string:** contains a number of useful constants and classes, as well as some deprecated legacy functions that are also available as methods on strings.

- **collections:** Implements specialized container datatypes providing alternatives to Python's general purpose built-in containers: dict, list, set, and tuple.

- **operator:** Exports a set of functions corresponding to the intrinsic operators of Python.

- **math:** Provides access to the mathematical functions defined by the C standard.

- **matplotlib:** A Python 2D plotting library which produces publication quality figures in a variety of hardcopy formats and interactive environments across platforms.

- **numpy:** The fundamental package for scientific computing with Python.

- **re:** Provides regular expression matching operations similar to those found in Perl.
- **pandas:** An open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.
- **glob:** Finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order.
- **zipfile:** Allows for handling of zipfiles.

Notice we import some of the packages differently. In some cases we just import the entire package when we say `import XYZ`. For some packages which are small, or, from which we are going to use a lot of the functionality it provides, this is fine. 

Sometimes when we import the package directly we say `import XYZ as X`. All this does is allow us to type `X` instead of `XYZ` when we use certain functions from the package. So we can now say `X.function()` instead of `XYZ.function()`. This saves time typing and eliminates errors from having to type out longer package names. I could just as easily type `import XYZ as potato` and whenever I use a function from the `XYZ` package I would need to type `potato.function()`. What we import the package as is up to you, but some commonly used packages have abbreviations that are standard amongst Python users such as `import pandas as pd` or `import matplotlib.pyplot as plt`. You do not need to us `pd` or `plt`, however, these are widely used and using something else could confuse other users and is generally considered bad practice. 

Other times we import only specific elements or functions from a package. This is common with packages that are very large and provide a lot of functionality, but from which we are only using a couple functions or a specific subset of the package that contains the functionality we need. This is seen when we say `from XYZ import ABC`. This is saying I only want the `ABC` function from the `XYZ` package. Sometimes we need to point to the specific location where a function is located within the package. We do this by adding periods in between the directory names, so it would look like `from XYZ.123.A1B2 import LMN`. This says we want the `LMN` function which is located in the `XYZ` package and then the `123` and `A1B2` directory in that package. 

You can also import more than one function from a package by separating the functions with commas like this `from XYZ import ABC, LMN, QRS`. This imports the `ABC`, `LMN` and `QRS` functions from the `XYZ` package.

In [2]:
from nltk.corpus import stopwords
import string
from collections import defaultdict
import operator
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
import glob
import zipfile

### Set needed variables
This is where you will make some decisions about your data and set the necessary variables. We are doing this so you will not need to make changes in the code further down.

First we assign the file type our data comes in to a variable. At the moment the only options are '.csv' or '.json' as these are the most popular twitter data formats. We assign the format to the `fileType` variable. It should look like this: `fileType = ".json"`.

Your data can be contained in one file or many. Therefore, you need to state if you want to read in a single document or an entire directory. If you want to read in a single document, then set `singleDoc` equal to **True**. If you want to read in an entire directory of documents then set `singleDoc` equal to **False**.

The `nltkStop` is where you determine if you want to use the built in stopword list provided by the NLTK package. They provide stopword lists in multiple languages. If you wish to use this then set `nltkStop` equal to **True**. If you do not, then set `nltkStop` equal to **False**.

The `customStop` variable is for if you have a dataset that contains additional stopwords that you would like to read in and have added to the existing `stopWords` list. You do **NOT** need to use the NLTK stopwords list in order to add your own custom list of stopwords. **NOTE: Your custom stopwords file needs to have one word per line as it reads in a line at a time and the full contents of the line is read in and added to the existing stopwords list.** If you have a list of your own then set `customStop` equal to **True**. If you do not have your own custom stopwords list then set `customStop` equal to **False**.

The `stopLang` variable is to choose the language of the nltk stopword list you wish to use. It is currently set to `"english"`. If you need a different language, simply change `"english"` to the anglicized name of the language you wish to use (e.g. "spanish" instead of "espanol" or "german" instead of "deutsch").

The variable `encoding` is where you determine what type of encoding to use (ascii, ISO-8850-1, utf-8, etc...). We have it set to utf-8 at the moment as we have found it is less likely to error. If there are errors, then your data may have elements with a different encoding. We have found that changing the encoding to `"ISO-8859-1"` often resolves the error, however, it may not properly encode certain characters, especially those with diacritical marks (à, á, â, ä, et cetera). You need to decide what impact this may have on your results.

The `stopWords = []` is an empty list that will contain the final list of stop words to be removed form your dataset. What ends up in the list depends on whether you set `nltkStop` and/or `customStop` equal to **True** or **False** and if you add any additional words to the list.

In [3]:
fileType = ".json"
singleDoc = True
nltkStop = True
customStop = True
stopLang = "english"
encoding = 'utf-8'
stopWords = []

#### File paths
Here we are saving different file paths as variables that we need in our code. We again do this so you will not need to make as many changes to the code later. 

First we use the `os` package above to find our `['HOME']` directory using the `environ` function. This will work for any operating system, so if you decide to try this out on your personal computer instead of ReD, the `homePath` variable will still be the path to your 'home' directory, so no changes are needed.

Next, we combine the `homePath` file path with the folder names that lead to where our data is stored. Note that we do not use any file names yet, just the path to the folder. This is because we may want to read in all the files in the directory, or just one, or we may need to access a file in this directory, but need to navigate to another folder to access other data. There are options below for doing both. We save the path as a variable named `dataHome`.

Now we add the `homePath` file path to other folder names that lead to a folder where we will want to save any output generated by this code. We again will change the file names for the output in the appropriate cells down below. We save this file path as the variable `dataResults`.

Lastly, we use the file path from `dataHome` to point to the directory where our data is stored and save that as `dataRoot`. Notice though that this is dependent upon whether you chose `".json"` or `".csv"` as your `fileType` above.

In [4]:
homePath = os.environ['HOME']
dataHome = os.path.join(homePath, "Text-Analysis-master", "data")
dataResults = os.path.join(homePath, "Text-Analysis-master", "Output")
if fileType == ".csv":
    dataRoot = os.path.join(dataHome, "twitter", "CSV")
else:
    dataRoot = os.path.join(dataHome, "twitter", "JSON")

### Stopwords
If you set `nltkStop` equal to **True** above then this will add the nltk stopwords list to the empty list named `stopWords`. You should already have chosen your language above, so there is no need to do that here.

If you wish to add additional words to the `stopWords` list, add the word in quotes to the list in `stopWords.extend(['the', 'words', 'you', 'want', 'to', 'add'])`.

In [5]:
# NLTK Stop words
if nltkStop is True:
    stopWords.extend(stopwords.words(stopLang))
    
    stopWords.extend(["coronavirus", "com", "pic", "twitter"])

#### Add own stopword list

Here is where your own stopwords list is added if you selected **True** in `customStop` above. Here you will need to change the folder names and file name to match your folders and file. Remember to put each folder name in quotes and in the correct path order, always putting the file name including the file extension ('.txt') last.

In [6]:
if customStop is True:
    stopWordsFilepath = os.path.join(dataHome, "twitterStopword.txt")

    with open(stopWordsFilepath, "r",encoding = encoding) as stopfile:
        stopWordsCustom = [x.strip() for x in stopfile.readlines()]

    stopWords.extend(stopWordsCustom)

### Functions
We need to create a few functions in order to calculate and create a bargraph. Any time you see `def` that means we are *DE*claring a *F*unction. The `def` is usually followed by the name of the function being created and then in parentheses are the parameters the function requires. After the parentheses is a colon, which closes the declaration, then a bunch of code below which is indented. The indented code is the program statement or statements to be executed. Once you have created your function all you need to do in order to run it is call the function by name and make sure you have included all the required parameters in the parentheses. This allows you to do what the function does without having to write out all the code in the function every time you wish to perform that task.

#### Text Cleaning
The first function does some text cleaning for us and requires the parameter: text (as in what text are you cleaning).

Now we come to the statements to be executed. First we lowercase the text or else 'Love' and 'love' will be counted as two different words, so we make them all 'love'. Next we remove URLs by removing any text that starts with 'http' and ending with a space. Then we split the text into individual words. Next we remove any empty space, digits, stopwords, and punctuation. Finally, we return a list of cleaned words.

In [7]:
def textClean(text):
    
    text = text.strip().lower()
    
    tweets = re.sub(r"http\S+", "", text)
    
    tokens = re.split(r'\W+', tweets )
    
    # remove empty string
    tokens = [t for t in tokens if t]
    
    # remove digits
    tokens = [t for t in tokens if not t.isdigit()]
    
    # built-in stop words list
    tokens = [t for t in tokens if t not in stopWords]
        
    # remove punctuation
    puncts = list(string.punctuation)
    puncts.append('--')

    tokens = [t for t in tokens if t not in puncts]

    return tokens

### Unzip files

Here we are unzipping files. Since twitter data can be rather large it is often necessary to compress it into a '.zip' file in order to upload it to places such as GitHub. For this reason, we have setup some code to go in and automatically extract all the items in a compressed '.zip' file so you don't have to and so you don't get errors later. If the data is not in a '.zip' file there is no need to worry, it will not give an error if there are no files ending in '.zip' in your directory.

You should not need to make any changes as we use the same variables containing our file paths as above, so if you need to make adjustments to the file paths, you need to make them there, specifically to the `dataRoot` variable.

In [8]:
direct = dataRoot
allZipFiles = glob.glob(os.path.join(dataRoot, "*.zip"))
for item in allZipFiles:
    fileName = os.path.splitext(direct)[0]
    zipRef = zipfile.ZipFile(item, "r")
    zipRef.extractall(fileName)
    zipRef.close()
    os.remove(item)

#### Read in a file

This function gives parameters and statements for reading in a file. First notice we have "DEclared" our "Function" using the `def` command. We have named the function `readTweets` and have given it four parameters, namely a filepath(`filepath`), what header our tweet content is found under (`textColIndex`), and what encoding to use (`encoding = encoding`). Notice the `encoding` and parameter is set to the variable we assigned in an earlier code cell.

Next we have an `if...else` statement. This statement says "if" we assigned `".csv"` to the `fileType` variable in an earlier code cell then run the line of code directly below this, if it is not `".csv"` (`else`) do this other line of code instead.

Then we convert the desired column to a list and assign that list to the variable `content`.

Lastly, the function returns the `content` variable when applied.

In [9]:
def readTweets(filepath, textColIndex, encoding = encoding):
    if fileType == ".csv":
        tweet = pd.read_csv(filepath, index_col=None, header =0, encoding = encoding, lineterminator='\n')
    else:
        tweet = pd.read_json(filepath, encoding = encoding)
    
    content = tweet[textColIndex].tolist()
    
    return content[1 : ]

#### Frequency count

Now we need a function to count all the words and put them in order from most frequent to least frequent. The first four lines say that we are creating a function that creates a dictionary that adds '1' next to a word in that dictionary every time that word is used. The last line says that we are sorting that dictionary from the highest number to the lowest number.

In [10]:
def getFreq(tokens):
    
    freq = defaultdict(int)

    for t in tokens:
        freq[t] += 1
    
    # sorted frequency in descending order
    return sorted(freq.items(), key = operator.itemgetter(1), reverse = True)

#### Plot Graph Function

This next function describes how we want to plot or visualize our results. We name this function `plotTopTen` and it begins by pulling only the top words from our sorted frequency above. You will be able to adjust the number of plotted words further down in another cell. After you choose how many words to plot, we put them in a list called `topNWords`. Next we assign the length of `topNWords` to the variable `x_pos` as this will be used to determine our x axis. Then we assign the count of each word to the variable `cnts` as this will be used for our y axis. Then it says we want a barplot (`plt.bar`) and the rest describes different aspects of the barplot, such as what we want for the x and y axis, the centering of the graph, the colors of the bars, the labels and titles of the graph, and how to save an image file of the visualization so we can use it in future presentations or publications. We will make changes to some of these later in the code.

In [11]:
def plotTopTen(sortedFreq, title, imgFilepath, dpi):
    
    topn = n

    for t in sortedFreq[0 : topn]:
        
        print('{} : {}'.format(t[0], t[1]))
    
    topNWords = [w for w in sortedFreq[0 : topn]]

    x_pos = np.arange(len(topNWords))
    cnts = [w[1] for w in topNWords]

    plt.rcdefaults()
    plt.figure(dpi=dpi, figsize = figSz)

    plt.bar(x_pos, cnts, align = 'center', alpha = 0.5, color = color)
    

        
    plt.xticks(x_pos, [w[0] for w in topNWords])
    plt.xticks(rotation = angle)
        
    xlabel = plt.xlabel('Words')
    xlabel.set_color(labCol)
    ylabel = plt.ylabel('Frequency')
    ylabel.set_color(labCol)
    
    high = max(cnts)
    low = 0
    
    plt.ylim(low, math.ceil(high + 0.1 * (high - low)))
    
    for xpos, count in zip(x_pos, cnts):
    
        plt.text(x = xpos, y = count + 1, s = str(count), ha = 'center', va = 'bottom')

    plt.title(title)
 
    plt.savefig(imgFilepath, format = fmt, dpi = dpi, bbox_inches = 'tight')
    
    plt.show()

#### Pull from a single file

This function reads a single file from which we will be counting and plotting the bargraph. This function calls the `readTweets` function form above, then adds the new line mark to the end of each row (`'\n'.join(map(str, content))`), so that the cleaning function we created above knows where the end of each line is. It then calls the `textClean` function to clean the text and returns the cleaned text.

In [12]:
def getTokensFromSingleText(dataFilepath, textColIndex, encoding):
    
    content = readTweets(dataFilepath, textColIndex, encoding)
    
    text = '\n'.join(map(str, content))

    return textClean(text)

#### Pull from a directory

This function reads all files in a directory from which we will be counting and plotting a bargraph. This function 'walks' through the directory and all sub-directories, ignores hidden files which are designated by the file name beginning with a period. The function then assigns the entire file path for each file in the directory to the variable `dataFilepath`. Then the function calls the `readTweets` function from above, adds the new line mark to the end of each row (`text = '\n'.join(map(str, content))`), so that the cleaning function we created above knows where the end of each line is. It then calls the `textClean` function we created above. Then the function says to print the file path as soon as the file has been processed so we can see the progress and finally it returns the newly cleaned tokens.

In [13]:
def getTokensFromScan(dataRoot, textColIndex, encoding):
    tokens = []
    
    for root, subdirs, files in os.walk(dataRoot):
        
        for filename in files:
            
            # skip hidden file
            if filename.startswith('.'):
                continue
            
            dataFilepath = os.path.join(root, filename)
            
            content = readTweets(dataFilepath, textColIndex, encoding)
            text = '\n'.join(map(str, content))
            tokens.extend(textClean(text))
                
            print('Finished tokenizing text {}\n'.format(dataFilepath))
            
    return tokens

### Plot Top Ten

Here we have an 'if else' statement that differentiates between creating a bargraph from a single document or an entire directory, depending on if you set `singleDoc` above equal to **True** or **False**.

#### First set variables
Before we run the cell below, make sure the variables in the beginning of the cell are set correctly. The `n` variable should have a number assigned to it that is the number of words you wish to plot in your graph.

First we need to determine the number of top words we wish to plot. We do this by assigning the number of words to the variable `n`. It is currently set to see the top 10 words, but you can raise or lower this number depending on your needs.

Then we assign our `textColIndex`, and `encoding` variables. The only one you will most likely want to change is the `textColIndex` variable. This will tell both the `plotTopTen` function which column in your data to process. You can either name it by index number (remembering to start counting at 0 and not 1) for '.csv' or by the column header for '.json' files. To do it by index number you count from the left beginning with 0 and enter the number of the column you wish to process. If you want the first column, put 0, if the third column, put 2, and so on. If you wish to choose the column by the header, then put the header in quotes and set `textColIndex` equal to that header. It should look like this `textColIndex = 'text'`.

The `singleDocName` variable should contain the single file you wish to run if you are only interested in a single document. If you set `singleDoc` above to **False** then don't worry about the `singleDocName` variable. This variable automatically adds the value of the `fileType` variable from above to the end of the document name. So if you chose '.csv' as your `fileType` and you have a file named 'myFile.csv', you only need to put 'myFile' in the quotes. The same goes for '.json' files.

Next we name the output '.svg' file produced by the code below. Right now the name is a generic `"topTenTwitter.svg"` but you may wish to change it to something that matches your data.

Then we set our file format. We assign this format to the variable `fmt`. Make sure the `fmt` matches the file type in the output file name.

Now we set our dpi. The `dpi` variable gives the desired resolution of our saved image file.

Next we choose our figure size and assign it to the variable `figSz`. The numbers in the parentheses are ordered (width, height).

Then we determine at what angle we want the x tick labels to be. The angle can be anywhere from 0 to 360. We assign this number to the variable `angle`.

Next, we need to give our graph a title. We assign the title to the variable `title`.

Lastly, we need to choose some color(s) for our graph. First we choose the bar colors. This is currently set so each bar has it's own color. If you choose to do this know that the colors in the list will repeat in the order you have them if you have more data points than you have colors in your list. The other option is to list only one color and make the bars all the same color. Either way, we assign our color selection as the variable `color`. Then we choose the color for our x axis and y axis labels. We assign our chosen color to the variable `labCol`.

#### From a single text

If you set `singleDoc` above equal to **True**, then the first part of the script in the box below uses the functions and file paths we have created to get word frequencies and plot a bar graph from a single '.txt' file. 

We start by pointing to the specific file. Then we break the file down into tokens uisng the single file function from above.

Next we count word frequencies using our word frequency function we made earlier.

Then we assign the file path where we are going to save our output image to the variable `imgFilepath`. 

Then we implement the plotting function we created above to give us a bargraph of the most frequent thematic words in our chosen text. 

#### Bargraph from multiple files in a directory

If you set `singleDoc` equal to **False**, then the second part of the script in the box below after `else:` now uses the functions and file paths we have created above to get word frequencies and plot a bargraph from multiple files in a directory. 

We start by breaking the documents in our folder down into tokens using the function for getting tokens from a directory of documents we made earlier.

Then we get the word frequencies using our function for counting words from above.

Then we assign the file path where we are going to save our output image to the variable `imgFilepath`.  

Then we implement the plotting function we created above to give us a bargraph of the most frequent thematic words in our chosen corpus. 

In [14]:
n = 10
textColIndex = "text"
singleDocName = 'coronaVirusFeb01-082020' + fileType
outputFile = "topTenTwitter.svg"
fmt = 'svg'
dpi = 1200
figSz = (10,5)
angle = 60
title = 'Top 10 Words, #Coronavirus'
color = ['crimson','orange', 'yellow', 'green', 'blue','darkorchid', 'darkred', 'darkorange','gold', 'darkgreen']
labCol = 'crimson'

if singleDoc is True:
    # Use case one, analyze top 10 most frequent words from a single text

    dataFilepath = os.path.join(dataRoot, singleDocName)

    # get tokens
    tokens = getTokensFromSingleText(dataFilepath, textColIndex, encoding)

    # get frequency
    freq = getFreq(tokens)

    imgFilepath = os.path.join(dataResults, outputFile)

    plotTopTen(freq, title, imgFilepath, dpi)
else:
    # Use case two, analyze top 10 most frequent words from a corpus root

    tokens = getTokensFromScan(dataRoot, textColIndex, encoding)

    # get frequency
    freq = getFreq(tokens)

    imgFilepath = os.path.join(dataResults, outputFile)

    plotTopTen(freq, title, imgFilepath, dpi)


china : 4143
wuhan : 1781
virus : 1567
people : 1296
outbreak : 995
flu : 978
chinese : 971
confirmed : 901
deaths : 860
health : 727


### VOILA!